## About iPython Notebooks ##

iPython Notebooks are interactive coding environments embedded in a webpage. You will be using iPython notebooks in this class. Make sure you fill in any place that says `# BEGIN CODE HERE #END CODE HERE`. After writing your code, you can run the cell by either pressing "SHIFT"+"ENTER" or by clicking on "Run" (denoted by a play symbol). Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

 **What you need to remember:**

- Run your cells using SHIFT+ENTER (or "Run cell")
- Write code in the designated areas using Python 3 only
- Do not modify the code outside of the designated areas
- In some cases you will also need to explain the results. There will also be designated areas for that.

Fill in your **NAME** and **AEM** below:

In [ ]:
NAME = "Alexia Ntantouri"
AEM = "3871"

---

# Assignment 3 - Ensemble Methods #

Welcome to your third assignment. This exercise will test your understanding on Ensemble Methods.

In [ ]:
# Always run this cell
import numpy as np
import pandas as pd

# USE THE FOLLOWING RANDOM STATE FOR YOUR CODE
RANDOM_STATE = 42

## Download the Dataset ##
Download the dataset using the following cell or from this [link](https://github.com/sakrifor/public/tree/master/machine_learning_course/EnsembleDataset) and put the files in the same folder as the .ipynb file.
In this assignment you are going to work with a dataset originated from the [ImageCLEFmed: The Medical Task 2016](https://www.imageclef.org/2016/medical) and the **Compound figure detection** subtask. The goal of this subtask is to identify whether a figure is a compound figure (one image consists of more than one figure) or not. The train dataset consits of 4197 examples/figures and each figure has 4096 features which were extracted using a deep neural network. The *CLASS* column represents the class of each example where 1 is a compoung figure and 0 is not.


In [ ]:
import urllib.request
url_train = 'https://github.com/sakrifor/public/raw/master/machine_learning_course/EnsembleDataset/train_set.csv'
filename_train = 'train_set.csv'
urllib.request.urlretrieve(url_train, filename_train)
url_test = 'https://github.com/sakrifor/public/raw/master/machine_learning_course/EnsembleDataset/test_set_noclass.csv'
filename_test = 'test_set_noclass.csv'
urllib.request.urlretrieve(url_test, filename_test)

('test_set_noclass.csv', <http.client.HTTPMessage at 0x7f89a6088370>)

In [ ]:
# Run this cell to load the data
train_set = pd.read_csv("train_set.csv").sample(frac=1).reset_index(drop=True)
train_set.head()
X = train_set.drop(columns=['CLASS'])
y = train_set['CLASS'].values

In [ ]:
!pip install -U imbalanced-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


The following code will reduce the number of instances, dealing with the small imbalance of the dataset, as well as reducing the size of the dataset!

In [ ]:
from collections import Counter
from imblearn.under_sampling import NeighbourhoodCleaningRule, RandomUnderSampler

ncr = NeighbourhoodCleaningRule()
X_res, y_res = ncr.fit_resample(X, y)
rus = RandomUnderSampler(random_state=42)
X_res, y_res = rus.fit_resample(X_res, y_res)
print('Resampled dataset shape %s' % Counter(y_res))
X = X_res
y = y_res

Resampled dataset shape Counter({0: 1687, 1: 1687})


## 1.0 Testing different ensemble methods ##
In this part of the assignment you are asked to create and test different ensemble methods using the train_set.csv dataset. You should use **5-fold cross validation** for your tests and report the average f-measure weighted and balanced accuracy of your models. You can use [cross_validate](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html#sklearn.model_selection.cross_validate) and select both metrics to be measured during the evaluation.

### !!! Use n_jobs=-1 where is posibble to use all the cores of a machine for running your tests ###

### 1.1 Voting ###
Create a voting classifier which uses two **simple** estimators/classifiers. Test both soft and hard voting and report the results. Consider as simple estimators the following:


*   Decision Trees
*   Linear Models
*   KNN Models

In [ ]:
### BEGIN SOLUTION
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_validate, cross_val_score
from sklearn.metrics import make_scorer, accuracy_score, f1_score

# USE RANDOM STATE!
cls1 = DecisionTreeClassifier(max_depth=5, random_state=RANDOM_STATE) # Classifier #1
cls2 = LogisticRegression(max_iter=1000, random_state=RANDOM_STATE) # Classifier #2
soft_vcls = VotingClassifier([('dt', cls1), ('lr', cls2)], voting='soft', n_jobs=-1) # Voting Classifier
hard_vcls = VotingClassifier([('dt', cls1), ('lr', cls2)], voting='hard', n_jobs=-1) # Voting Classifier

metrics = ['f1_weighted', 'balanced_accuracy']

svlcs_scores = cross_validate(soft_vcls, X, y, cv=5, scoring=metrics) # return_train_score=True
s_avg_fmeasure = np.mean(svlcs_scores['test_f1_weighted']) # The average f-measure
s_avg_accuracy = np.mean(svlcs_scores['test_balanced_accuracy']) # The average accuracy

hvlcs_scores = cross_validate(hard_vcls, X, y, cv=5, scoring=metrics)
h_avg_fmeasure = np.mean(hvlcs_scores['test_f1_weighted']) # The average f-measure
h_avg_accuracy = np.mean(hvlcs_scores['test_balanced_accuracy']) # The average accuracy

### END SOLUTION

print("Classifier:")
print(soft_vcls)
print("F1 Weighted-Score: {} & Balanced Accuracy: {}".format(round(s_avg_fmeasure,4), round(s_avg_accuracy,4)))

print("Classifier:")
print(hard_vcls)
print("F1 Weighted-Score: {} & Balanced Accuracy: {}".format(round(h_avg_fmeasure,4), round(h_avg_accuracy,4)))

Classifier:
VotingClassifier(estimators=[('dt',
                              DecisionTreeClassifier(max_depth=5,
                                                     random_state=42)),
                             ('lr',
                              LogisticRegression(max_iter=1000,
                                                 random_state=42))],
                 n_jobs=-1, voting='soft')
F1 Weighted-Score: 0.882 & Balanced Accuracy: 0.882
Classifier:
VotingClassifier(estimators=[('dt',
                              DecisionTreeClassifier(max_depth=5,
                                                     random_state=42)),
                             ('lr',
                              LogisticRegression(max_iter=1000,
                                                 random_state=42))],
                 n_jobs=-1)
F1 Weighted-Score: 0.8295 & Balanced Accuracy: 0.8316


For both soft/hard voting classifiers the F1 weighted score should be above 0.74 and 0.79, respectively, and for balanced accuracy 0.74 and 0.80. Remember! This should be the average performance of each fold, as measured through cross-validation with 5 folds!

### 1.2 Randomization

You are asked to create three ensembles of decision trees where each one uses a different method for producing homogeneous ensembles. Compare them with a simple decision tree classifier and report your results in the dictionaries (dict) below using as key the given name of your classifier and as value the f1_weighted/balanced_accuracy score. The dictionaries should contain four different elements. Use the same cross-validation approach as before!

In [ ]:
### BEGIN SOLUTION
from sklearn.ensemble import StackingClassifier, BaggingClassifier, AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier

ens1 = BaggingClassifier(n_jobs=-1, random_state=RANDOM_STATE) # n_estimators=1000
ens2 = GradientBoostingClassifier(random_state=RANDOM_STATE)
ens3 = RandomForestClassifier(n_jobs=-1, random_state=RANDOM_STATE) # n_estimators=100, max_depth=10
tree = DecisionTreeClassifier(max_depth=5, random_state=RANDOM_STATE)

f_measures = dict()
accuracies = dict()
# Example f_measures = {'Simple Decision':0.8551, 'Ensemble with random ...': 0.92, ...}

metrics = ['f1_weighted', 'balanced_accuracy']
models = {'Decision Tree': tree, 'Bagging': ens1, 'Gradient Boosting': ens2, 'Random Forest': ens3}

for model_name in models:
    scores = cross_validate(models[model_name], X, y, scoring=metrics, cv=5)
    avg_fmeasure = np.mean(scores['test_f1_weighted']) # The average f-measure
    avg_accuracy = np.mean(scores['test_balanced_accuracy']) # The average accuracy
    f_measures[model_name] = avg_fmeasure #type(model).__name__
    accuracies[model_name] = avg_accuracy

### END SOLUTION

print(ens1)
print(ens2)
print(ens3)
print(tree)
for name,score in f_measures.items():
    print("Classifier: {} -  F1 Weighted: {}".format(name,round(score,4)))
for name,score in accuracies.items():
    print("Classifier: {} -  BalancedAccuracy: {}".format(name,round(score,4)))

BaggingClassifier(n_jobs=-1, random_state=42)
GradientBoostingClassifier(random_state=42)
RandomForestClassifier(n_jobs=-1, random_state=42)
DecisionTreeClassifier(max_depth=5, random_state=42)
Classifier: Decision Tree -  F1 Weighted: 0.7603
Classifier: Bagging -  F1 Weighted: 0.808
Classifier: Gradient Boosting -  F1 Weighted: 0.8633
Classifier: Random Forest -  F1 Weighted: 0.8638
Classifier: Decision Tree -  BalancedAccuracy: 0.7605
Classifier: Bagging -  BalancedAccuracy: 0.8083
Classifier: Gradient Boosting -  BalancedAccuracy: 0.8634
Classifier: Random Forest -  BalancedAccuracy: 0.864


### 1.3 Question

Increasing the number of estimators in a bagging classifier can drastically increase the training time of a classifier. Is there any solution to this problem? Can the same solution be applied to boosting classifiers?

YOUR ANSWER HERE

A solution to this problem could be parallelization. Bagging can be parallelized since each estimator is trained independently on a subset of the data.

On the other hand boosting classifiers do not support parallelization. Unlike bagging, boosting algorithms build an ensemble sequentially, where each subsequent model learns from the mistakes of the previous models. So the boosting classifiers cannot be trained in parallel.

## 2.0 Creating the best classifier ##
In the second part of this assignment, we will try to train the best classifier, as well as to evaluate it using stratified cross valdiation.

### 2.1 Good Performing Ensemble

In this part of the assignment you are asked to train a good performing ensemble, that is able to be used in a production environment! Describe the process you followed to achieve this result. How did you choose your classifier and your parameters and why. Report the f-measure (weighted) & balanced accuracy, using 10-fold stratified cross validation, of your final classifier. Can you achieve a balanced accuracy over 88%, while keeping the training time low? (Tip 1: You can even use a model from the previous parts, but you are advised to test additional configurations, and ensemble architectures, Tip 2: If you try a lot of models/ensembles/configurations or even grid searches, in your answer leave only the classifier you selected as the best!)

In [ ]:
### BEGIN SOLUTION
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier

best_cls = XGBClassifier(n_estimators=100,tree_method='gpu_hist')

metrics = ['f1_weighted', 'balanced_accuracy']

skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=RANDOM_STATE)
best_scores = cross_validate(best_cls, X, y, cv=skf, scoring=metrics)

best_fmeasure = np.mean(best_scores['test_f1_weighted'])
best_accuracy = np.mean(best_scores['test_balanced_accuracy'])

### END SOLUTION

print("Classifier:")
#print(best_cls)
print("F1 Weighted-Score: {} & Balanced Accuracy: {}".format(best_fmeasure, best_accuracy))

Classifier:
F1 Weighted-Score: 0.8935725992614175 & Balanced Accuracy: 0.8936073541842774


LEAVE HERE ANY COMMENTS ABOUT YOUR CLASSIFIER

XGBoost is one of the most powerful boosting algorithms. It provides high accuracy and it is very fast.

### 2.2 Question
What other ensemble architectures you tried, and why you did not choose them as your final classifier?

YOUR ANSWER HERE

First, I tried random forests which are very popular in classification problems but I wasn't able to achieve the required score. I used random search to find the best parameters but the highest balanced accuracy I managed to get was 87%.

Then I tried gradient boosting which produces accurate models. But I didn't choose it as my final classifier because gradient boosting models can be computationally expensive and take a long time to train.

In the end, I tried XGBoost which is a powerful algorithm that provides high accuracy and speed. I achieved over 88% balanced accuracy while keeping the training time low. So this is the classifier I selected as the best.

### 2.3 Setup the Final Classifier
Finally, in this last cell, set the cls variable to either the best model as occured by the stratified cross_validation, or choose to retrain your classifier in the whole dataset (X, y). There is no correct answer, but try to explain your choice. Then, save your model using pickle and upload it with your submission to e-learning!

In [ ]:
import pickle

### BEGIN SOLUTION
best_cls = XGBClassifier(n_estimators=100,tree_method='gpu_hist')

best_cls.fit(X,y)

# save with pickle
file_name = "xgb_classification.pkl"
pickle.dump(best_cls, open(file_name, "wb"))
### END SOLUTION


# load
cls = pickle.load(open(file_name, "rb"))

test_set = pd.read_csv("test_set_noclass.csv")
predictions = cls.predict(test_set)

# We are going to run the following code
if False:
  from sklearn.metrics import f1_score, balanced_accuracy_score
  final_test_set = pd.read_csv('test_set.csv')
  ground_truth = final_test_set['CLASS']
  print("Balanced Accuracy: {}".format(balanced_accuracy_score(predictions, ground_truth)))
  print("F1 Weighted-Score: {}".format(f1_score(predictions, ground_truth, average='weighted')))

Both metrics should aim above 82%! This is going to be tested by us! Make sure your cross validation or your retrained model achieves high balanced accuracy and f1_score (based on 2.1) (more than 88%) as it should achieve at least 82% in our unknown test set!


COMMENT

I retrained my classifier in the whole dataset (X,y) because the purpose of cross-validation is model checking, not model building.

Please provide your feedback regarding this project! Did you enjoy it?

In [ ]:
# YOUR ANSWER HERE

# Yes, I enjoyed it. There was more freedom to experiment with different models and that was fun.